### Setup (Installing Libraries)

In [ ]:
! pip install langchain_community tiktoken langchain-groq langchain-huggingface langchainhub chromadb langchain langgraph tavily-python

In [ ]:
GROQ_API_KEY = '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'
TAVILY_API_KEY = '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'

In [ ]:
#create index
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 250, chunk_overlap = 0
)

doc_splits = text_splitter.split_documents(docs_list)

vectorstore = Chroma.from_documents(
    documents=doc_splits,
    embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-l6-v2"),
    collection_name='rag-chroma'
)
retriever = vectorstore.as_retriever()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Retrieval Grader

from langchain.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq

class GradeDocuments(BaseModel):
  """Binary score for checking relevance on retrieved documents."""
  binary_score: str = Field(
      description="Documents are relevant to the question,'yes' or 'no'"
  )

# LLM

llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model='llama-3.3-70b-versatile',
    temperature=0
)

structured_llm_grader = llm.with_structured_output(GradeDocuments)

#promt

system = """You are a grader assessing relevance of a retrieved document to a user question. \n
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

grade_prompt= ChatPromptTemplate.from_messages(
    [
        ("system",system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}")
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader
question = "agent memory"
docs = retriever.get_relevant_documents(question)
doc_txt=docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

binary_score='yes'


In [ ]:
# generate

from langchain import hub
from langchain_core.output_parsers import StrOutputParser

#prompt

prompt = hub.pull("rlm/rag-prompt")

#llm

llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model='llama-3.3-70b-versatile',
    temperature=0
)

#post-processing

def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

#chain
rag_chain = prompt | llm | StrOutputParser()

#run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


An agent's memory consists of two types: short-term memory, which utilizes in-context learning to learn, and long-term memory, which retains and recalls information over extended periods using an external vector store and fast retrieval. The long-term memory allows the agent to retain and recall infinite information. The memory stream is a long-term memory module that records a comprehensive list of agents' experience in natural language.


In [ ]:
#Question Re-writer

#llm

llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model='llama-3.3-70b-versatile',
    temperature=0
)

#prompt

system = """You a question re-writer that converts an input question to a better version that is optimized \n
     for web search. Look at the input and try to reason about the underlying semantic intent / meaning."""

re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        )
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

'Based on the input "agent memory", I\'ll attempt to reason about the underlying semantic intent. It appears to be related to artificial intelligence or computer science. Here\'s a reformulated question:\n\n"What is agent memory in artificial intelligence, and how does it impact agent learning and decision-making?"\n\nAlternatively, if the context is more specific to a particular field, such as psychology or neuroscience, the question could be:\n\n* "How does memory function in intelligent agents, such as humans or animals?"\n* "What role does memory play in agent-based modeling and simulation?"\n\nHowever, if I had to choose a more general and web-search-friendly question, I\'d suggest:\n\n"Agent memory in AI: definition, types, and applications"\n\nThis revised question is more specific, concise, and optimized for web search, allowing users to find relevant information on the topic.'

### Web Search Tool (Using TAVILY)

In [ ]:
# Web Search Tool

from langchain_community.tools.tavily_search import TavilySearchResults

import os
os.environ["TAVILY_API_KEY"] = "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"

web_search_tool = TavilySearchResults(k=3)

### Creating Graph

In [ ]:
# Define graph state

from typing import List

from typing_extensions import TypedDict

class GraphState(TypedDict):
  """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        web_search: whether to add search
        documents: list of documents
    """

  question: str
  generation: str
  web_search: str
  documents: List[str]

In [ ]:
from langchain.schema import Document

def retrieve(state):
  """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
  print("---RETRIEVE---")
  question = state['question']

  #Retrieval
  documents = retriever.get_relevant_documents(question)
  return {"documents": documents, "question": question }

def generate(state):
  """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
  print("---GENERATE---")
  question = state["question"]
  documents = state["documents"]

  #RAG Generation
  generation = rag_chain.invoke({"context": documents, "question": question})
  return {"documents": documents, "question": question, "generation": generation}

def grade_documents(state):
  """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """
  print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
  question = state["question"]
  documents = state["documents"]

  # Score each doc
  filtered_docs = []
  web_search = "No"
  for d in documents:
    score = retrieval_grader.invoke(
        {"question": question, "document": d.page_content}
    )
    grade = score.binary_score
    if grade == "yes":

      print("---GRADE: DOCUMENT RELEVANT---")
      filtered_docs.append(d)
    else:
      print("---GRADE: DOCUMENT NOT RELEVANT---")
      web_search = "Yes"
      continue
  return {"documents": filtered_docs, "question": question, "web_search": web_search}

def transform_query(state):
  """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """
  print("---TRANSFORM QUERY---")
  question = state['question']
  documents = state['documents']

  #Re-write question
  better_question = question_rewriter.invoke({"question": question})
  return {"documents": documents, "question": better_question}

from langchain.schema import Document

def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    question = state["question"]
    documents = state["documents"]

    # Web search
    docs = web_search_tool.invoke({"query": question})

    # Check if docs is a list of strings or a list of dictionaries
    if isinstance(docs, list) and all(isinstance(d, dict) and "content" in d for d in docs):
        web_results = "\n".join([d["content"] for d in docs])
    else:
        # If docs is a list of strings (assuming docs is just raw content)
        web_results = "\n".join(docs)

    # Create Document and append to documents list
    web_results = Document(page_content=web_results)
    documents.append(web_results)

    return {"documents": documents, "question": question}


def decide_to_generate(state):
  """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """
  print("---ASSESS GRADED DOCUMENTS---")
  state["question"]
  web_search = state["web_search"]
  state["documents"]

  if web_search=="Yes":
    print(
        "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
    )
    return "transform_query"
  else:
    print("---DECISION: GENERATE---")
    return "generate"

## Compile Graph

In [ ]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
workflow.add_node("transform_query", transform_query)  # transform_query
workflow.add_node("web_search_node", web_search)  # web search

# Build graph
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "web_search_node")
workflow.add_edge("web_search_node", "generate")
workflow.add_edge("generate", END)

# Compile
app = workflow.compile()

### Use The Graph

In [ ]:
from pprint import pprint

# Run
inputs = {"question": "What are the types of agent memory?"}
for output in app.stream(inputs):
  for key, value in output.items():
    pprint(f"Node '{key}':" )
  pprint(("\n--\n"))
#final generation
pprint(value["generation"])

---RETRIEVE---
"Node 'retrieve':"
'\n--\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---
"Node 'grade_documents':"
'\n--\n'
---TRANSFORM QUERY---
"Node 'transform_query':"
'\n--\n'
---WEB SEARCH---
"Node 'web_search_node':"
'\n--\n'
---GENERATE---
"Node 'generate':"
'\n--\n'
('The two main types of memory models used in artificial intelligence agents '
 'are short-term memory and long-term memory. Short-term memory utilizes '
 'in-context learning, while long-term memory provides the capability to '
 'retain and recall information over extended periods, often by leveraging an '
 'external vector store and fast retrieval. Additionally, a memory stream, '
 'which is a long-term memory module, can record a comprehensive list of '
 "agents' experience in natural 

In [ ]:
from pprint import pprint

# Run
inputs = {"question": "How does the AlphaCodium paper work?"}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---
"Node 'grade_documents':"
'\n---\n'
---TRANSFORM QUERY---
"Node 'transform_query':"
'\n---\n'
---WEB SEARCH---
"Node 'web_search_node':"
'\n---\n'
---GENERATE---
"Node 'generate':"
'\n---\n'
("I don't know the key findings and methodologies presented in the AlphaCodium "
 'research paper. The provided context does not mention AlphaCodium or any '
 'related research paper. It appears to be related to prompt engineering and '
 'language models, but does not contain relevant information about '
 'AlphaCodium.')
